## PROJETO ETL ENEM
___
#### Responder às seguintes perguntas:
1. Qual a escola com a maior média de notas?
2. Qual o aluno com a maior média de notas e o valor dessa média?
3. Qual a média geral?
4. Qual o % de Ausentes?
5. Qual o número total de Inscritos?
6. Qual a média por disciplina?
7. Qual a média por Sexo?
8. Qual a média por Etnia?

In [1]:
%pip install pyspark > /dev/null
%pip install mysql-connector-python > /dev/null
%pip install findspark > /dev/null
%pip install sqlalchemy > /dev/null
%pip install pandas > /dev/null
%pip install setuptools > /dev/null
%pip install findspark > /dev/null
 

Note: you may need to restart the kernel to use updated packages.


O sistema n o pode encontrar o caminho especificado.


Note: you may need to restart the kernel to use updated packages.


O sistema n o pode encontrar o caminho especificado.


Note: you may need to restart the kernel to use updated packages.


...

O sistema n o pode encontrar o caminho especificado.


Note: you may need to restart the kernel to use updated packages.


O sistema n o pode encontrar o caminho especificado.


Note: you may need to restart the kernel to use updated packages.


O sistema n o pode encontrar o caminho especificado.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import findspark
from pyspark.sql import functions as f
import os
import mysql.connector
from uteis.ETL import ETL
from uteis.load_to_database import load_to_mysql
from uteis.answer import Answer

In [3]:
findspark.init()

In [4]:
os.environ["HADOOP_HOME"] = r"C:\hadoop-3.3.6"
os.environ["hadoop.home.dir"] = r"C:\hadoop-3.3.6"

In [5]:
spark = SparkSession.builder \
    .appName("ENEM ETL") \
    .config("spark.driver.memory", "8g")\
    .config("spark.jars", "mysql-connector-java-8.0.26.jar") \
    .getOrCreate()

In [6]:
spark

In [7]:
etl = ETL()

## ETL

### EXTRACT

In [8]:

file_path = r"..\database\raw\MICRODADOS_ENEM_2020.csv"
df = etl.extract(spark, file_path)

In [9]:
df.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [10]:
df.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- NU_ANO: string (nullable = true)
 |-- TP_FAIXA_ETARIA: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_NACIONALIDADE: string (nullable = true)
 |-- TP_ST_CONCLUSAO: string (nullable = true)
 |-- TP_ANO_CONCLUIU: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- TP_ENSINO: string (nullable = true)
 |-- IN_TREINEIRO: string (nullable = true)
 |-- CO_MUNICIPIO_ESC: string (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: string (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: string (nullable = true)
 |-- TP_LOCALIZACAO_ESC: string (nullable = true)
 |-- TP_SIT_FUNC_ESC: string (nullable = true)
 |-- CO_MUNICIPIO_PROVA: string (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: string (nullable = tru

### TRANSFORM

In [11]:
dim_aluno, dim_escola, dim_municipio, fato_enem = etl.transform(df)

In [12]:
dim_aluno.show()

+----------------+----+-----+------------------+
|numero_inscricao|sexo|etnia|situacao_conclusao|
+----------------+----+-----+------------------+
|    200005909927|   F|    0|                 1|
|    200006123011|   F|    1|                 2|
|    200006267793|   F|    1|                 1|
|    200003990748|   F|    2|                 1|
|    200004009640|   F|    3|                 1|
|    200002371686|   M|    3|                 1|
|    200004571138|   M|    3|                 1|
|    200004046927|   M|    1|                 1|
|    200003051097|   F|    2|                 1|
|    200005983749|   F|    3|                 2|
|    200003885791|   F|    3|                 1|
|    200002065330|   M|    1|                 1|
|    200001510845|   M|    2|                 1|
|    200005221474|   F|    2|                 1|
|    200004604317|   F|    1|                 2|
|    200006546014|   M|    3|                 2|
|    200005366948|   M|    3|                 2|
|    200001246495|  

In [13]:
dim_escola.show()

+----------------+--------------------+------+
|codigo_municipio|       nom_municipio|estado|
+----------------+--------------------+------+
|         1508209|               Vigia|    PA|
|         2411205|          Santa Cruz|    RN|
|         2801306|              Capela|    SE|
|         4209607|        Lauro Müller|    SC|
|         3304300|          Rio Bonito|    RJ|
|         2609709|               Orobó|    PE|
|         3157203|       Santa Bárbara|    MG|
|         2403608|            Extremoz|    RN|
|         1505809|              Portel|    PA|
|         2505006|              Cubati|    PB|
|         3203700|        Muniz Freire|    ES|
|         5210000|             Inhumas|    GO|
|         4322202|         Tupanciretã|    RS|
|         1505486|              Pacajá|    PA|
|         2928901|       São Desidério|    BA|
|         3108255|     Bonito de Minas|    MG|
|         1304237|           Tonantins|    AM|
|         4105706|         Clevelândia|    PR|
|         522

In [14]:
dim_municipio.show()

+----------------+--------------------+------+
|codigo_municipio|       nom_municipio|estado|
+----------------+--------------------+------+
|         1508209|               Vigia|    PA|
|         2411205|          Santa Cruz|    RN|
|         2801306|              Capela|    SE|
|         4209607|        Lauro Müller|    SC|
|         3304300|          Rio Bonito|    RJ|
|         2609709|               Orobó|    PE|
|         3157203|       Santa Bárbara|    MG|
|         2403608|            Extremoz|    RN|
|         1505809|              Portel|    PA|
|         2505006|              Cubati|    PB|
|         3203700|        Muniz Freire|    ES|
|         5210000|             Inhumas|    GO|
|         4322202|         Tupanciretã|    RS|
|         1505486|              Pacajá|    PA|
|         2928901|       São Desidério|    BA|
|         3108255|     Bonito de Minas|    MG|
|         1304237|           Tonantins|    AM|
|         4105706|         Clevelândia|    PR|
|         522

In [15]:
fato_enem.show()

+----------------+----------------+-------+-------+-------+-------+------------+------------------+-----------+-----------+-----------+-----------+
|numero_inscricao|codigo_municipio|nota_CN|nota_CH|nota_LC|nota_MT|nota_redacao|        NOTA_MEDIA|presenca_CN|presenca_CH|presenca_LC|presenca_MT|
+----------------+----------------+-------+-------+-------+-------+------------+------------------+-----------+-----------+-----------+-----------+
|    200006271946|            NULL|   NULL|   NULL|   NULL|   NULL|        NULL|              NULL|          0|          0|          0|          0|
|    200001195856|            NULL|  604.1|  661.7|  595.3|  711.3|         580|            630.48|          1|          1|          1|          1|
|    200001943954|         2927408|   NULL|   NULL|   NULL|   NULL|        NULL|              NULL|          0|          0|          0|          0|
|    200001908998|         3547304|  620.8|  675.0|  624.2|  759.4|         760| 687.8799999999999|          1| 

### LOAD

In [16]:
parquet_path = r"..\database\transformed\dados_enem_parquet"

etl.load(dim_aluno, dim_escola, dim_municipio, fato_enem, parquet_path)

### RESPONDENDO QUESTÕES

In [17]:
answer = Answer(spark, dim_aluno, dim_escola, dim_municipio, fato_enem)

In [18]:

# load answer
top_school = answer.get_top_school()
top_student = answer.get_top_student()
average_score = answer.get_average_score()
absent_percentage = answer.get_absent_percentage()
total_students = answer.get_total_students()
average_by_discipline = answer.get_average_by_discipline()
average_by_gender = answer.get_average_by_gender()
average_by_ethnicity = answer.get_average_by_ethnicity()

##### 1. Qual a escola com a maior média de notas?

In [19]:
top_school.show()

+-------------------+------+-----------------+
|             escola|estado|     media_escola|
+-------------------+------+-----------------+
|Santana do Manhuaçu|    MG|706.5600000000001|
+-------------------+------+-----------------+



##### 2. Qual o aluno com a maior média de notas e o valor dessa média?


In [20]:
top_student.show()

+----------------+----+-----+-----------------+
|numero_inscricao|sexo|etnia|      media_notas|
+----------------+----+-----+-----------------+
|    200005996961|   M|    1|858.5799999999999|
+----------------+----+-----+-----------------+



##### 3. Qual a média geral?

In [21]:
average_score.show()

+-----------------+
|      media_geral|
+-----------------+
|526.5806802769449|
+-----------------+



##### 4. Qual o % de Ausentes?


In [22]:
absent_percentage.show()

+-------------------+
|percentual_ausentes|
+-------------------+
|  55.20821067007383|
+-------------------+



##### 5. Qual o número total de Inscritos?


In [23]:
total_students.show()

+---------------+
|total_inscritos|
+---------------+
|        5783109|
+---------------+



##### 6. Qual a média por disciplina?


In [24]:
average_by_discipline.show()

+----------------+-----------------+-----------------+-----------------+-----------------+
|        media_CN|         media_CH|         media_LC|         media_MT|    media_redacao|
+----------------+-----------------+-----------------+-----------------+-----------------+
|490.534635128853|514.2869822121747|526.0192024818806|520.7272992307672|581.3352823310404|
+----------------+-----------------+-----------------+-----------------+-----------------+



##### 7. Qual a média por Sexo?


In [25]:
average_by_gender.show()

+----+-----------------+
|sexo|       media_sexo|
+----+-----------------+
|   F|521.2449112081382|
|   M|534.7254885958835|
+----+-----------------+



##### 8. Qual a média por Etnia?

In [26]:
average_by_ethnicity.show()

+-----+------------------+
|etnia|       media_etnia|
+-----+------------------+
|    1|   556.52672213979|
|    3| 508.6575016996888|
|    5|472.84366774078046|
|    4| 524.9384386151191|
|    2|  500.868301996394|
|    0|       534.0503396|
+-----+------------------+



#### CONECTAR AO SQL

##### PERSISTIR OS DADOS

In [ ]:
load_to_mysql(dim_aluno, dim_escola, dim_municipio, fato_enem)

In [ ]:
spark.stop()